In [ ]:
import warnings

warnings.filterwarnings("ignore")

import os
import pathlib
import numpy as np
import pandas as pd
import yfinance as yf
from dotenv import load_dotenv
from rich import print
from rich.console import Console

In [75]:
# display tweaks
# Set Pandas to display float values with 4 decimal places
pd.options.display.float_format = "{:.4f}".format
pd.options.display.max_colwidth = 1024  # make wide before wrapping
pd.options.display.max_rows = 100  # before truncation

# Set NumPy to display float values with 4 decimal places
np.set_printoptions(precision=4, suppress=True)


# load all API keys for LLMs
load_dotenv()
console = Console()

In [71]:
def is_valid_ticker(symbol: str) -> bool:
    """
    Checks if symbol is a valid ticker symbol. For it to be valud, yf.Ticker(symbol).info
    should not raise an exception!

    Args:
        symbol(str): a ticker symbol (such as "AAPL" or "PERSISTENT.NS")
        (please visit Yahoo Finance website to get valid symbol of company)
    """
    try:
        ticker = yf.Ticker(symbol)
        return "shortName" in ticker.info
    except Exception as e:
        return False

In [72]:
tickers = ["AAPL", "MSFT", "RELIANCE.NS", "PIDILITE.NS", "MADEUPCO.NS"]
for ticker in tickers:
    console.print(
        f"{ticker}: is_valid? {'[green]Yes[/green]' if is_valid_ticker(ticker) else '[red]No[/red]'}"
    )

AAPL: is_valid? Yes

MSFT: is_valid? Yes

RELIANCE.NS: is_valid? Yes

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PIDILITE.NS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PIDILITE.NS&crumb=8jGkjmCy.PG


PIDILITE.NS: is_valid? No

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MADEUPCO.NS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MADEUPCO.NS&crumb=8jGkjmCy.PG


MADEUPCO.NS: is_valid? No

In [73]:
# dummy cell, so that I can see the keys of each element when coding
ticker = yf.Ticker("RELIANCE.NS")
financials = ticker.financials.transpose().sort_index(ascending=True)
balance_sheet = ticker.balance_sheet.transpose().sort_index(ascending=True)
cash_flow = ticker.cashflow.transpose().sort_index(ascending=True)
income_stmt = ticker.income_stmt.transpose().sort_index(ascending=True)

In [ ]:
def liquidity_ratios(ticker_symbol: str) -> pd.DataFrame:
    """
    Calculates the following end-of-financial-year liquidity ratios
        - Current Ratio = Current Assets / Current Liabilities
        - Quick Ratio = (Current Assets - Inventory) / Current Liabilities
        - Cash Ratio = (Cash & Equivalents) / Current Liabilities

    Liquidity ratios help determine short-term stability of a company. These can be interpreted
    as follows (though the interpretation could differ person-to-person)
        - Current Ratio: > 1.5 is good (indicates ability to cover short-term liabilities). < 1 risky!
        - Quick Ratio: > 1 is good (measures liquidity excluding inventory) [conservative number]
        - Cash Ratio: Company's pure cash position/pile - higher the better.

    Args:
        ticker_symbol (string) - ticker symbol of company (as on Yahoo! Finance)
            (e.g., AAPL for Apple, AMZN for Amazon, RELIANCE.NS for Reliance Industries)

    Returns:
        a pandas dataframe, with datetime index of financial year-end dates
        and calculated liquidity ratios as columns

    Example:
        ```python
        import yfinance as yf
        import pandas as pd

        # replace RELIANCE.NS with appropriate company symbol as used by Yahoo! Finance
        # (e.g., AAPL for Apple, AMZN for Amazon, COLPAL.NS for Colgate Palmolive on NSE)
        df = liquidity_ratios("RELIANCE.NS")
        print(df)
        ```

        ```bash
        # above code will display something like this
                            Current Ratio Quick Ratio Cash Ratio
            2020-03-31           NaN         NaN        NaN
            2021-03-31        1.3439      1.0496     0.0523
            2022-03-31        1.1243      0.7751     0.1086
            2023-03-31        1.0747      0.7209     0.0863
            2024-03-31        1.1830      0.7986     0.2354
        ```
    """
    # ticker_data, financials, balance_sheet, cash_flow = fetch_data(ticker_symbol)
    ticker = yf.Ticker(ticker_symbol)
    balance_sheet = ticker.balance_sheet.transpose().sort_index(ascending=True)

    current_assets = balance_sheet["Current Assets"]
    current_liabilities = balance_sheet["Current Liabilities"]
    # some companies may not report inventory (e.g. Reliance does, Persistent does not)
    inventory_fields_exist = "Inventory" in ticker.balance_sheet.index

    ratios = {}
    ratios["Current Ratio"] = current_assets / current_liabilities
    if inventory_fields_exist:
        ratios["Quick Ratio"] = (
            current_assets - balance_sheet["Inventory"]
        ) / current_liabilities

    ratios["Cash Ratio"] = (
        balance_sheet["Cash And Cash Equivalents"]
        / balance_sheet["Current Liabilities"]
    )

    return pd.DataFrame(ratios)

In [39]:
print(liquidity_ratios("RELIANCE.NS"))

Current Ratio Quick Ratio Cash Ratio
2020-03-31           NaN         NaN        NaN
2021-03-31        1.3439      1.0496     0.0523
2022-03-31        1.1243      0.7751     0.1086
2023-03-31        1.0747      0.7209     0.0863
2024-03-31        1.1830      0.7986     0.2354

In [ ]:
def profitability_ratios(ticker_symbol: str) -> pd.DataFrame:
    """
    Calculates the following end-of-financial-year profitability ratios
        - Return on Equity (RoE) = Net Income / Shareholder's Equity
        - Return on Assets (RoA) = Net Income / Total Assets
        - Return on Capital Employed (RoCE) = EBIT / (Total Assets - Current Liabilities)
        - Net Profit Margin = Net Income / Revenue'
        - Operating Margin = Operating Income / Revenue

    Financial ratios help earnings of and returns from a company. These can be interpreted
    as follows (though the interpretation could differ person-to-person)
        - Return on Equity (RoE) - >15% is good (higher return on shareholder investment)
        - Return on Assets (RoA) - >5% is good (shows how efficiently assets generate capital)
        - Return on Capital Employed (RoCE) - >12% preferred, measures efficiency in capital utilization
        - Net Profit Margin - >10% is strong, measures final profitability per dollar of revenue
        - Operating Margin - >20% is Excellent (strong profitability & cost efficiency),
            10-20% is Good (healthy operations & stable profits), 5-10% (Low, businesses is operational, but struggles with cost), <5% is weak (thin margins & potential weakness), negative (loss making business!!)

    Args:
        ticker_symbol (string) - ticker symbol of company (as on Yahoo! Finance)
            (e.g., AAPL for Apple, AMZN for Amazon, RELIANCE.NS for Reliance Industries)

    Returns:
        a pandas dataframe, with datetime index of financial year-end dates
        and calculated profitability ratios as columns

    Example:
        ```python
        import yfinance as yf
        import pandas as pd

        # replace RELIANCE.NS with appropriate company symbol as used by Yahoo! Finance
        # (e.g., AAPL for Apple, AMZN for Amazon, COLPAL.NS for Colgate Palmolive on NSE)
        df = profitability_ratios"RELIANCE.NS")
        print(df)
        ```
    """
    ticker = yf.Ticker(ticker_symbol)
    balance_sheet = ticker.balance_sheet.transpose().sort_index(ascending=True)
    financials = ticker.financials.transpose().sort_index(ascending=True)
    income_stmt = ticker.income_stmt.transpose().sort_index(ascending=True)

    revenue = financials["Total Revenue"]
    operating_income = financials["Operating Income"]
    net_income = financials["Net Income"]
    total_assets = balance_sheet["Total Assets"]
    shareholder_equity = balance_sheet["Stockholders Equity"]
    ebit = income_stmt["EBIT"]
    current_liabilities = balance_sheet["Current Liabilities"]

    ratios = {}
    ratios["Return on Equity (RoE)"] = net_income / shareholder_equity
    ratios["Return on Assets (RoA)"] = net_income / total_assets
    ratios["Return on Capital Employed (RoCE)"] = ebit / (
        total_assets - current_liabilities
    )
    ratios["Net Profit Margin"] = net_income / revenue
    ratios["Operating Margin"] = operating_income / revenue

    return pd.DataFrame(ratios)

In [76]:
print(profitability_ratios("RELIANCE.NS"))

Return on Equity (RoE) Return on Assets (RoA)  \
2020-03-31                    NaN                    NaN   
2021-03-31                 0.0702                 0.0372   
2022-03-31                 0.0779                 0.0405   
2023-03-31                 0.0932                 0.0415   
2024-03-31                 0.0877                 0.0396   

           Return on Capital Employed (RoCE) Net Profit Margin  \
2020-03-31                               NaN               NaN   
2021-03-31                            0.0703            0.1052   
2022-03-31                            0.0811            0.0872   
2023-03-31                            0.0933            0.0760   
2024-03-31                            0.0931            0.0773   

           Operating Margin  
2020-03-31              NaN  
2021-03-31           0.1160  
2022-03-31           0.1132  
2023-03-31           0.1162  
2024-03-31           0.1239

In [ ]:
def efficiency_ratios(ticker_symbol: str) -> pd.DataFrame:
    """
    Calculates the following end-of-financial-year efficiency ratios
        - Asset Turnover Ratio = Revenue / Total Assets
        - Inventory Turnover = Cost of Goods Sold (COGS) / Inventory

    Financial ratios measure operational efficiency of a company. These can be interpreted
    as follows (though the interpretation could differ person-to-person)
        - Asset Turnover Ratio - measures how effectively assets generate sales. Higher is better
        - Inventory Turnover - >5 means fast-moving stocks/inventory, lower values indicate excess inventory
            (NOTE: not all companies report inventory!)

    Args:
        ticker_symbol (string) - ticker symbol of company (as on Yahoo! Finance)
            (e.g., AAPL for Apple, AMZN for Amazon, RELIANCE.NS for Reliance Industries)

    Returns:
        a pandas dataframe, with datetime index of financial year-end dates
        and calculated efficiency ratios as columns
    """
    ticker = yf.Ticker(ticker_symbol)
    balance_sheet = ticker.balance_sheet.transpose().sort_index(ascending=True)
    financials = ticker.financials.transpose().sort_index(ascending=True)

    ratios = {}

    revenue = financials["Total Revenue"]
    cost_of_goods_sold = financials["Cost Of Revenue"]
    total_assets = balance_sheet["Total Assets"]

    # Inventory Data
    # NOTE: inventory may or may not get reported. For example,
    # Tata Motors reports it, Persisteny Systems does not
    inventory_fields_exist = "Inventory" in ticker.balance_sheet.index
    if inventory_fields_exist:
        inventory = balance_sheet["Inventory"]
        average_inventory = inventory.rolling(2).mean()

    ratios["Asset Turnover"] = revenue / total_assets
    if inventory_fields_exist:
        ratios["Inventory Turnover"] = cost_of_goods_sold / average_inventory

    return pd.DataFrame(ratios)

In [46]:
print(efficiency_ratios("RELIANCE.NS"))

Asset Turnover Inventory Turnover
2020-03-31            NaN                NaN
2021-03-31         0.3534                NaN
2022-03-31         0.4641             5.6129
2023-03-31         0.5461             5.4232
2024-03-31         0.5131             4.6083

In [ ]:
def valuation_ratios(ticker_symbol: str) -> pd.DataFrame:
    """
    Calculates the following end-of-financial-year valuation ratios
        - Price-to-Earnings Ratio (P/E) - Price per Share / Earnings per Share (EPS)
        - Price-to-Sales Ratio (P/S) - Market Capitalization / Revenue
        - Price-to-Book Ratio (P/B) - Market Capitalization / Book Value of Equity
        - EV/EBIDTA ratio - (Market Cap + Debt - Cash & Equivalents) / EBIDTA
            where EV = Enterprise value & EBIDTA = Earnings before interest, taxes,
            depreciation and amortization

    Valuation ratios are intended to measure stock price "fairness" - generally we should
    look for lower values, which means stock price is undervalues, so should be a good investement.
    These can be interpreted as follows (though the interpretation could differ person-to-person)
        - Price-to-Earnings Ratio (P/E) - <20 is reasonable, lower means undervalued (so good for
            investment), higher suggests overvalues (to be avoided)
        - Price-to-Sales Ratio (P/S) - <2 means potentially undervalued (should target), 2-4 fair valued,
            and >4 potentially overvalues (should be avoided)
        - Price-to-Book Ratio (P/B) - <3 is good. Implies undervalued compared to book value (so
            good as an investment target)
        - EV/EBIDTA ratio - <10 is preferred (= cheap or undervalued, so good investment target),
            10-15 (= fair value - neutral) and >15 (= expensive or overvalued - avoid!)

    Args:
        ticker_symbol (string) - ticker symbol of company (as on Yahoo! Finance)
            (e.g., AAPL for Apple, AMZN for Amazon, RELIANCE.NS for Reliance Industries)

    Returns:
        a pandas dataframe, with datetime index of financial year-end dates
        and calculated profitability ratios as columns
    """
    ticker = yf.Ticker(ticker_symbol)
    balance_sheet = ticker.balance_sheet.transpose().sort_index(ascending=True)
    financials = ticker.financials.transpose().sort_index(ascending=True)

    ratios = {}

    market_cap = ticker.info["marketCap"]
    revenue = financials["Total Revenue"]
    shareholder_equity = balance_sheet["Stockholders Equity"]
    ebidta = financials.get(
        "EBIDTA",
        financials["Operating Income"]
        + financials.get("Depreciation & Amortization", 0),
    )
    total_debt = balance_sheet["Total Debt"]
    cash_equivalents = balance_sheet["Cash And Cash Equivalents"]
    ev = market_cap + total_debt - cash_equivalents

    ratios["Price-to-Earnings (P/E)"] = ticker.info["trailingPE"]
    ratios["Price-to-Sales (P/S)"] = market_cap / revenue
    ratios["Price-to-Book (P/B)"] = market_cap / shareholder_equity
    ratios["EV/EBIDTA"] = ev / ebidta

    return pd.DataFrame(ratios)

In [48]:
print(valuation_ratios("RELIANCE.NS"))

Price-to-Earnings (P/E) Price-to-Sales (P/S) Price-to-Book (P/B)  \
2020-03-31                  24.4484                  NaN                 NaN   
2021-03-31                  24.4484               3.6222              2.4155   
2022-03-31                  24.4484               2.4301              2.1697   
2023-03-31                  24.4484               1.9266              2.3625   
2024-03-31                  24.4484               1.8770              2.1315   

           EV/EBIDTA  
2020-03-31       NaN  
2021-03-31   35.7592  
2022-03-31   24.6276  
2023-03-31   19.5218  
2024-03-31   17.4080

In [ ]:
def leverage_ratios(ticker_symbol: str) -> pd.DataFrame:
    """
    Calculates the following end-of-financial-year leverage ratios
        - Debt-to-Equity Ratio (D/E) - Total Debt / Shareholders Equity
        - Interest Coverage - EBIT / Interest Expense

    Leverage ratios are intended to measure debt risk of a company. These can be interpreted
    as follows (though the interpretation could differ person-to-person)
        - Debt-to-Equity Ratio (D/E) - <1 is good. Higher values means higher debt risk
        - Interest Coverage - >3 is safe, lower values means higher risk of default

    Args:
        ticker_symbol (string) - ticker symbol of company (as on Yahoo! Finance)
            (e.g., AAPL for Apple, AMZN for Amazon, RELIANCE.NS for Reliance Industries)

    Returns:
        a pandas dataframe, with datetime index of financial year-end dates
        and calculated profitability ratios as columns
    """
    ticker = yf.Ticker(ticker_symbol)
    balance_sheet = ticker.balance_sheet.transpose().sort_index(ascending=True)
    financials = ticker.financials.transpose().sort_index(ascending=True)

    total_debt = balance_sheet["Total Debt"]
    shareholder_equity = balance_sheet["Stockholders Equity"]
    ebit = financials["EBIT"]
    interest_expense = financials["Interest Expense"]

    ratios = {}
    ratios["Debt-to-Equity (D/E)"] = total_debt / shareholder_equity
    ratios["Interest Coverage"] = ebit / interest_expense

    return pd.DataFrame(ratios)

In [50]:
print(leverage_ratios("RELIANCE.NS"))

Debt-to-Equity (D/E) Interest Coverage
2020-03-31                  NaN               NaN
2021-03-31               0.3715            4.0972
2022-03-31               0.3617            6.6901
2023-03-31               0.4671            5.9608
2024-03-31               0.4362            5.8102

In [67]:
def performance_and_growth_metrics(ticker_symbol: str) -> pd.DataFrame:
    """
    Calculates the following end-of-financial-year performance & growth metrics
        - Revenue Growth (%) = (Current Year Revenue - Previous Year Revenue) / Previous Year Revenue
        - EBIT Growth (%) = (Current Year EBIT - Previous Year EBIT) / Previous Year EBIT
        - EPS Growth (%) = (Current Year EPS - Previous Year EPS) / Previous Year EPS
        - FCF Growth (%) = (Current Year FCF - Prev Year FCF) / Prev Year FCF [FCF = Free Cash Flow]
        - Net Profit Margin (%) = Net Income / Total Revenue
        - Earnings Per Share (EPS) = (Net Income - Preferred Dividends) / Shares Outstanding
        - Debt to Equity (D/E) = Total Debt / Total Shareholders' Equity
        - Free Cash Flow = Operating Cash Flow - Capital Expenditure

    Revenue & growth metrics are intended to measure business growth, profitability, earnings potential, financial stability, and cash flow generation of a company.
    These can be interpreted as follows (though the interpretation could differ person-to-person)
        - Revenue Growth (%) - measures revenue growth across years. >10% is strong growth,
            5-10% is moderate growth, <5% is slow growth, and negative is revenue decline (avoid!)
        - EBIT Growth (%) - measures profitabity growth across years. >10% is strong operational efficiency &
            profitability growth, 5-10% is moderate growth, <5% is slow growth and negative means declining
            profitability (avoid!). Positive values indicate company is becoming more profitable.
        - EPS Growth (%) - measures how fast the company's earnings are increasing on a per-share basis.
            >15% means strong EPS growth (great for investors), 5-15% means moderate but stable growth,
            0-5% means slow earnings expansion and <0% (negative) means declining EPS (avoid!)
        - FCF Growth (%) - shows whether company is generating more "excess cash flow" over time.
            >15% means strong cash flows (company can invest & pay dividends), 5-15% means moderate growth & stable cash generation, 0-5% means low growth (cash flow stagnation) and negative means declining FCF (bad for investment - avoid!)
        - Net Profit Margin (%) - measures the profitability of a company.
            >10% means healthy profitability, 5-10% means sustainable profitability, 0-5% means thin margins, and negative means loss making business (avoid!)
        - Earnings Per Share (EPS) - measures profit earned by the company per share.
            Increasing EPS means company is growing profits per share - look for Y-on-Y values.
            Declining EPS means EPS is shrinking & this company should be avoided.
        - Debt to Equity (D/E) - measures debt-burden risk for a company
            <1.0 means healthy leverage (debt burden is low, hence good)
            1.0 - 2.0 usually means moderate risk, but varies by industry
            > 2.0 implies high debt burden (risky & should be avoided!)
        - Free Cash Flow - measures cash flows in & out of the company
            a positive value indicates good financial health (income > expenses)
            a negative value indicates excessive spending or low profitability of company

    Args:
        ticker_symbol (string) - ticker symbol of company (as on Yahoo! Finance)
            (e.g., AAPL for Apple, AMZN for Amazon, RELIANCE.NS for Reliance Industries)

    Returns:
        a pandas dataframe, with datetime index of financial year-end dates
        and calculated profitability ratios as columns
    """
    ticker = yf.Ticker(ticker_symbol)
    balance_sheet = ticker.balance_sheet.transpose().sort_index(ascending=True)
    financials = ticker.financials.transpose().sort_index(ascending=True)
    cash_flow = ticker.cash_flow.transpose().sort_index(ascending=True)

    ratios = pd.DataFrame(index=financials.index)
    ratios["Revenue Growth (%)"] = financials["Total Revenue"].pct_change() * 100.0
    ratios["EBIT Growth (%)"] = financials["EBIT"].pct_change() * 100.0
    ratios["Net Profit Margin (%)"] = (
        financials["Net Income"] / financials["Total Revenue"]
    ) * 100.0
    shares_outstanding = balance_sheet["Ordinary Shares Number"]
    eps = financials["Net Income"] / shares_outstanding
    ratios["EPS Growth (%)"] = eps.pct_change() * 100.0

    ratios["EPS"] = financials["Net Income"] / shares_outstanding
    ratios["Debt-to-Equity"] = (
        balance_sheet["Total Debt"] / balance_sheet["Stockholders Equity"]
    )
    ratios["Free Cash Flow"] = cash_flow["Free Cash Flow"]
    ratios["FCF Growth (%)"] = cash_flow["Free Cash Flow"].pct_change() * 100.0

    # NOTE: unlike other functions, ratios is a DataFrame
    return ratios

In [59]:
cash_flow.transpose()

,2020-03-31,2021-03-31,2022-03-31,2023-03-31,2024-03-31
Free Cash Flow,NaN,-788790000000.0000,105090000000.0000,-259560000000.0000,59050000000.0000
Repayment Of Debt,NaN,-872400000000.0000,-406470000000.0000,-290590000000.0000,-350550000000.0000
Issuance Of Debt,NaN,332110000000.0000,593430000000.0000,359360000000.0000,696100000000.0000
Issuance Of Capital Stock,NaN,132150000000.0000,397670000000.0000,400000000.0000,70000000.0000
Capital Expenditure,NaN,-1058370000000.0000,-1001450000000.0000,-1409880000000.0000,-1528830000000.0000
End Cash Position,NaN,173970000000.0000,361780000000.0000,686640000000.0000,972250000000.0000
Other Cash Adjustment Outside Changein Cash,NaN,240000000.0000,9410000000.0000,-17660000000.0000,7200000000.0000
Beginning Cash Position,NaN,309200000000.0000,173970000000.0000,361780000000.0000,686640000000.0000
Changes In Cash,NaN,-135470000000.0000,178400000000.0000,342520000000.0000,278410000000.0000
Financing Cash Flow,NaN,1019040000000.0000,172890000000.0000,104550000000.0000,-166460000000.0000


In [68]:
print(performance_and_growth_metrics("RELIANCE.NS"))

Revenue Growth (%)  EBIT Growth (%) Net Profit Margin (%)  \
2020-03-31                 NaN              NaN                   NaN   
2021-03-31                 NaN              NaN               10.5216   
2022-03-31             49.0527          31.6541                8.7224   
2023-03-31             26.1324          16.9911                7.5985   
2024-03-31              2.6462          11.9421                7.7265   

            EPS Growth (%)     EPS Debt-to-Equity     Free Cash Flow  \
2020-03-31             NaN     NaN            NaN                NaN   
2021-03-31             NaN 36.3262         0.3715 -788790000000.0000   
2022-03-31         23.4933 44.8604         0.3617  105090000000.0000   
2023-03-31          9.8773 49.2914         0.4671 -259560000000.0000   
2024-03-31          4.3759 51.4483         0.4362   59050000000.0000   

            FCF Growth (%)  
2020-03-31             NaN  
2021-03-31             NaN  
2022-03-31       -113.3229  
2023-03-31       -346.9883  
2024-03-31       -122.7500